Загружаем whisper и моедль

In [ ]:
! pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import whisper

model = whisper.load_model("large-v2")

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:65: UserWarning: /root/.cache/whisper/large-v2.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|█████████████████████████████████████| 2.87G/2.87G [00:36<00:00, 84.1MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_

In [ ]:
model.device

device(type='cuda', index=0)

Скачиваем 5 различных спикеров и подготавливаем по 2 фрагмента речи

In [ ]:
!git clone https://github.com/NoKorun/ASR_Systems

fatal: destination path 'ASR_Systems' already exists and is not an empty directory.


In [ ]:
! pip install pydub

In [ ]:
from pydub import AudioSegment
import os

In [ ]:
def make_audio_fragment(audio_name, startMin, startSec, endMin, endSec):
  startTime = startMin*60*1000+startSec*1000
  endTime = endMin*60*1000+endSec*1000

  song = AudioSegment.from_mp3('/content/ASR_Systems/'+audio_name+'.mp3')
  extract = song[startTime:endTime]

  # Saving
  if not os.path.isfile("/content/" + audio_name+'-fragment1.mp3'):
    extract.export( audio_name+'-fragment1.mp3', format="mp3")
  else:
    extract.export( audio_name+'-fragment2.mp3', format="mp3")

In [ ]:
def transcribe(audio):

    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    return result.text


# **Что Где Когда**

In [ ]:
from IPython.display import Audio
Audio("/content/ASR_Systems/www3.mp3")

In [ ]:
make_audio_fragment('www3', 0, 5, 0, 19)
make_audio_fragment('www3', 3, 10, 3, 29)

In [ ]:
Audio("/content/www3-fragment1.mp3")

In [ ]:
asr_text = transcribe("/content/www3-fragment1.mp3")
print(asr_text)

Detected language: ru
Фёдор, Фёдор Доминяев, объясните мне, пожалуйста, вот у вас такие сильные игроки, и вы всё время топчетесь в минуте на одном месте. Вы никуда не сдвигаетесь. Почему это происходит? Не знаю. Знали бы, не топтались бы.


Оригинальная транскрипция:

Фёдор, Фёдор Д***винятин***, объясните мне, пожалуйста, вот у вас такие сильные игроки, и вы всё время топчетесь в минуте на одном месте. Вы никуда не сдвигаетесь. Почему это происходит? Не знаю. Знали бы, не топтались бы.


Считаем Word Error Rate и Character Error Rate.


---



Word Error Rate = (S + D + I) / N, где

Есть некоторые опечатки, которые заменяются - S.

Иногда система вставляет слова, которые не были сказаны - I.

А некоторые слова удаляются, то есть вообще не воспринимаются - D.

N - количество слов в тексте.



---


Character Error Rate - то же самое, но уже с буквами.

В нашем случае

WER = (1+0+0)/36 = 0.028

CER = (7 + 0 + 0)/170 = 0.041

In [ ]:
Audio("/content/www3-fragment2.mp3")

In [ ]:
asr_text = transcribe("/content/www3-fragment2.mp3")
print(asr_text)

Detected language: ru
знать русский. Название этих трёх предметов в русском языке оканчивается на «со». Вот просо, вот мясо, вот колесо. Покажите колесо. Вот колесо, самое настоящее. А в чёрном ящике, очевидно, принадлежности для игры в «серсо». Ещё одно слово, которое заканчивается в русском языке на «со».


Оригинальная транскрипция:

Знать русский. Название этих трёх предметов в русском языке оканчивается на «со». Вот просо, вот мясо, вот колесо. Покажите колесо. Вот колесо, самое настоящее. А в чёрном ящике, очевидно, принадлежности для игры в «серсо». Ещё одно слово, которое заканчивается в русском языке на «со».

WER = (0)/44 = 0

CER = (0)/224 = 0

# **Коллапс**

In [ ]:
Audio("/content/ASR_Systems/collapse.mp3")

In [ ]:
make_audio_fragment('collapse', 0, 9, 0, 22)
make_audio_fragment('collapse', 0, 49, 1, 7)

In [ ]:
Audio("/content/collapse-fragment1.mp3")

In [ ]:
asr_text = transcribe("/content/collapse-fragment1.mp3")
print(asr_text)

Detected language: ru
от Тимирязевской ехал до Новодачной, а я где-то сидел рядом и слушал, он без конца трезвонил по телефону на весь вагон.


Оригинальная транскрипция:

От Тимирязевской ехал до Новодачной, а я где-то сидел рядом и слушал, он без конца трезвонил по телефону на весь вагон.

WER = (0)/21 = 0

CER = (0)/95 = 0

In [ ]:
Audio("/content/collapse-fragment2.mp3")

In [ ]:
asr_text = transcribe("/content/collapse-fragment2.mp3")
print(asr_text)

Detected language: ru
у него был коллапс. Вот это слово коллапс он произносил вот так вот изысканно, что весь вагон аж... коллапс! Новое слово!


Оригинальная транскрипция:

У него был коллапс. Вот это слово коллапс он произносил вот так вот изысканно, что весь вагон аж ***ах***! коллапс! Новое слово!

WER = (1)/22 = 0.046

CER = (2)/96 = 0.021

# **Пантеон**

In [ ]:
Audio("/content/ASR_Systems/pantheon.mp3")

In [ ]:
make_audio_fragment('pantheon', 0, 36, 0, 53)
make_audio_fragment('pantheon', 1, 38, 1, 53)

In [ ]:
Audio("/content/pantheon-fragment1.mp3")

In [ ]:
asr_text = transcribe("/content/pantheon-fragment1.mp3")
print(asr_text)

Detected language: ru
Я буду сражаться, пока не рухну на колени, пока скользкое от крови копье не выпадет из рук. Но и тогда вам не одолеть меня. Я плюну вам в лицо.


Оригинальная транскрипция:

Я буду сражаться, пока не рухну на колени, пока скользкое от крови копье не выпадет из рук. Но и тогда вам не одолеть меня. Я плюну вам в лицо.

WER = (0)/29 = 0

CER = (0)/110 = 0

In [ ]:
Audio("/content/pantheon-fragment2.mp3")

In [ ]:
asr_text = transcribe("/content/pantheon-fragment2.mp3")
print(asr_text)

Detected language: ru
Гора, на которую нужно взойти, не всегда высока. Это даже не всегда гора. Когда мы вдыхаем, мы должны помнить, что пьем предсмертные вздохи погибших до нас.


Оригинальная транскрипция:

Гора, на которую нужно взойти, не всегда высока. Это даже не всегда гора. Когда мы вдыхаем, мы должны помнить, что пьем предсмертные вздохи погибших до нас.

WER = (0)/26 = 0

CER = (0)/124 = 0

# **Брежнев**

In [ ]:
Audio("/content/ASR_Systems/brezhnev.mp3")

In [ ]:
make_audio_fragment('brezhnev', 0, 14, 0, 31)
make_audio_fragment('brezhnev', 1, 2, 1, 22)

In [ ]:
Audio("/content/brezhnev-fragment1.mp3")

In [ ]:
asr_text = transcribe("/content/brezhnev-fragment1.mp3")
print(asr_text)

Detected language: ru
Дорогие друзья, солдаты и матросы, сержанты и старшины, мичмены и немичмены,


Оригинальная транскрипция:

Дорогие друзья, солдаты и матросы, сержанты и старшины, мичм***е***ны и немичм***е***ны

WER = (0)/11 = 0

CER = (2)/62 = 0.032

In [ ]:
Audio("/content/brezhnev-fragment2.mp3")

In [ ]:
asr_text = transcribe("/content/brezhnev-fragment2.mp3")
print(asr_text)

Detected language: ru
пожелать всему личному составу наших вооруженных сил счастья, крепкого здоровья, больших успехов в боевой и политической подготовке системы ассиста.


Оригинальная транскрипция:

Пожелать всему личному составу наших вооруженных сил счастья, крепкого здоровья, больших успехов в боевой и политической подготовке систем***атически***.

WER = (1)/18 = 0.056

CER = (8)/127 = 0.063

# **Афордабл**

In [ ]:
Audio("/content/ASR_Systems/affordable.mp3")

In [ ]:
make_audio_fragment('affordable', 0, 17, 0, 29)
make_audio_fragment('affordable', 2, 29, 2, 47)

In [ ]:
Audio("/content/affordable-fragment1.mp3")

In [ ]:
asr_text = transcribe("/content/affordable-fragment1.mp3")
print(asr_text)

Detected language: ru
Во-первых, он великолепный человек. Его дизайн могут носить любого возраста, любого шейпа, размера, высоты. И он timeless.


Оригинальная транскрипция:

Во-первых, он великолепный человек. Его дизайн***с*** могут носить любого возраста, любого шейп, ***сайз***, ***хайт***. И он timeless.

WER = (3)/17 = 0.176

CER = (14)/93 = 0.151

In [ ]:
Audio("/content/affordable-fragment2.mp3")

In [ ]:
asr_text = transcribe("/content/affordable-fragment2.mp3")
print(asr_text)

Detected language: ru
Красивые цвета, все цвета, что есть, модные. Красный, черный, hot pink, apple green, citrus orange, mango colors, шелк очень модный. Очень модные пиджаки с какими-то details.


Оригинальная транскрипция:

Красивые цвета, все цвета, что есть, модные. Красный, черный, hot pink, apple green, citrus orange, mango colors, шелк очень модный. Очень модные пиджаки с какими-то details.

WER = (0)/26 = 0

CER = (0)/136 = 0

Итоговые результаты

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'Metric': ["WER1", "CER1", "WER2", "CER2"],
                   'ChKG': [0.028, 0.041, 0, 0],
                   'Collapse' : [0, 0, 0.046, 0.021],
                   'Pantheon': [0, 0, 0, 0],
                   'Brezhnev': [0, 0.032, 0.056, 0.063],
                   'Affordable': [0.176, 0.151, 0, 0]})

df

,Metric,ChKG,Collapse,Pantheon,Brezhnev,Affordable
0,WER1,0.028,0.000,0,0.000,0.176
1,CER1,0.041,0.000,0,0.032,0.151
2,WER2,0.000,0.046,0,0.056,0.000
3,CER2,0.000,0.021,0,0.063,0.000
